**Ma chu hang: HP, RW, GB**

**Kho xuat: kho tot**

In [1]:
import glob
import os
import openpyxl
from openpyxl import load_workbook
import numpy as np
import pandas as pd

# setting notebook
pd.set_option("display.max_columns", None)

In [15]:
%run 01-db.ipynb 

C:\Users\user\Desktop\gte\notebooks\data-joined
C:\Users\user\Desktop\gte\notebooks


In [16]:
# copy
po = po_run.copy()

In [17]:
# rename col
dict = { 'ID_PO': 'id',
         'ID_cus': 'id_account',
         'Mã chủ hàng': 'stakeholder',
         'date_receive': 'date_receive',
         'deli_dl': 'delivery_deadline',
         'Ngày thực tế giao hàng': 'delivery_date',
         'Chứng từ KH ký nhận': 'invoice_date',
         'Chứng từ khác': 'pickup_note',
         'Tình trạng đơn hàng': 'status',
         'Kho xuất': 'warehouse',
         'Loại phiếu': 'po_type',
         'month': 'month_receive',
         'datediff_chungtu': 'datediff_invoice',
         'datediff_giaohang': 'datediff_delivery',
         'cbm_po': 'cbm',
         'kg_po': 'weight_g',
         'unit_po': 'no_unit',
         'sku_po': 'no_sku',
         'amount_po': 'amount',
         'Region': 'region',
         'Account': 'account_name',
         'Des': 'destination',
         'in_sub': 'in_out_city',
         'day_level': 'day_level'
       }

# call rename () method
po.rename(columns=dict, inplace=True)

In [18]:
po.head(1)

,id,id_account,stakeholder,date_receive,delivery_deadline,delivery_date,invoice_date,pickup_note,status,warehouse,po_type,month_receive,datediff_invoice,datediff_delivery,cbm,weight_g,no_unit,no_sku,amount,region,channel,account_name,destination,in_out_city,day_level
0,SO-0301220007HP,FHSA305HP,HP,2022-01-03,2022-01-05,2022-01-10,2022-01-13,Đã nhặt hàng ngày 04/01/2022,Đã bàn giao chứng từ,Kho Phạm Văn Đồng,Đơn bán,1,3.0,7.0,0.0308,16956.0,86.0,5.0,1627294.0,Miền Nam,offline,FAHASA,Hồ Chí Minh,Nội thành,0 - 6 ngày


# invoice

In [20]:
# group
aggregations = {
    'id': 'count',
    'amount': 'mean',
    'cbm': 'median',
    'weight_g': 'median',
    'no_unit': 'median',
    'no_sku': 'median',
    
}
summary = po.groupby([
    'channel',
    'stakeholder',
    'month_receive',
    'status',
    'region',
    'day_level',
    'in_out_city',
    'po_type'], as_index=False, dropna=False).agg(aggregations)

summary

,channel,stakeholder,month_receive,status,region,day_level,in_out_city,po_type,id,amount,cbm,weight_g,no_unit,no_sku
0,offline,GB,1,Đã bàn giao chứng từ,Miền Bắc,Trễ,NaN,Đơn bán,4,1.183525e+07,0.0859,21737.75,118.5,11.0
1,offline,GB,1,Đã bàn giao chứng từ,Miền Bắc,Trễ,NaN,Đơn khác,2,NaN,NaN,NaN,NaN,NaN
2,offline,GB,1,Đã bàn giao chứng từ,Miền Nam,Trễ,NaN,Đơn bán,17,1.770203e+07,0.0568,16528.00,52.0,13.0
3,offline,GB,1,Đã bàn giao chứng từ,Miền Nam,Trễ,NaN,Đơn khác,1,NaN,NaN,NaN,NaN,NaN
4,offline,HP,1,Chưa thực hiện,Miền Bắc,nan,Nội thành,Đơn khác,2,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023,online,RW01,12,Đã bàn giao chứng từ,online,0 - 6 ngày,NaN,Đơn bán,2165,1.235527e+05,0.0016,1085.00,1.0,1.0
2024,online,RW01,12,Đã bàn giao chứng từ,online,7 - 14 ngày,NaN,Đơn bán,133,1.393815e+05,0.0016,1085.00,1.0,1.0
2025,online,RW01,12,Đã giao hàng,online,0 - 6 ngày,NaN,Đơn bán,275,1.316592e+05,0.0016,1085.00,1.0,1.0
2026,online,RW01,12,Đã giao hàng,online,15 - 30 ngày,NaN,Đơn bán,13,1.547554e+05,0.0004,141.67,1.0,1.0


In [21]:
summary.to_excel('../data/processed/summary-meeting-processed.xlsx', sheet_name='offline', index=False)